# Ejemplos de uso para el Lexer y Parser Implementa


Importar API de Gramática

In [18]:
from cmp.pycompiler import Grammar, Terminal, NonTerminal, Token

## Importar el Lexer

In [19]:
from Lexer_Parser.lexer import Lexer

## Importar el Parser LR1

In [20]:
from Lexer_Parser.shift_reduce import LR1Parser, evaluate_reverse_parse


## Importar AST

In [21]:
from cmp.ast import Node, BinaryNode,AtomicNode, UnaryNode, BinaryNode, ConstantNumberNode, DivNode, StarNode, MinusNode, PlusNode,  EqualNode, get_printer

### Crear Gramática

In [22]:
G = Grammar()  # Crear gramática

Añadir terminales y no-terminales

In [23]:
E = G.NonTerminal('E', True)
A = G.NonTerminal('A')
equal, plus, num = G.Terminals('= + int')

### Producciones

In [24]:
E %= num, lambda h, s: ConstantNumberNode(s[1]), None
E %= A + equal + A, lambda h, s: EqualNode(s[1], s[3]), None, None, None
A %= num + plus + A, lambda h, s: PlusNode(ConstantNumberNode(s[1]), s[3]), None, None, None
A %= num, lambda h, s: ConstantNumberNode(s[1]), None

#### Generar el Lexer

Se añade al array de tipos una tupla(tokenType,regex) y el G.EOF para decir que es $

In [25]:
lexer = Lexer(
    [
        (num, '(1|2|3|4|5|6|7|8|9)(0|1|2|3|4|5|6|7|8|9)*'),

        ('space', '( |\t|\n)( |\t|\n)*'),

        (equal, '='),
        (plus, '\+'),

    ], G.EOF
)

#### Generar el parser con la gramática

In [26]:
parser = LR1Parser(G)

# Función para sacar los datos interesantes:

In [27]:
 def parse(text:str):
    """
    Parsea la cadena, printea los token las producciones y las operaciones shift reduce adeams del ast
    :param text: 
    :return: 
    """
    all_tokens = lexer(text)
    tokens = list(filter(lambda token: token.token_type != 'space', all_tokens))
    print(f"Los tokens son {tokens}")
    right_parse, operations = parser(tokens)
    print(right_parse)
    print("-----------------------------")
    print(operations)
    print("###############################")
    
    ast = evaluate_reverse_parse(right_parse, operations, tokens)
    
    printer = get_printer(AtomicNode=ConstantNumberNode, BinaryNode=BinaryNode)
    print(f" \n El ast es: \n {printer(ast)}")

In [28]:
parse("6+7=13")

Los tokens son [int: 6, +: +, int: 7, =: =, int: 13, $: $]
[A -> int, A -> int + A, A -> int, E -> A = A]
-----------------------------
['SHIFT', 'SHIFT', 'SHIFT', 'REDUCE', 'REDUCE', 'SHIFT', 'SHIFT', 'REDUCE', 'REDUCE']
###############################
 
 El ast es: 
 \__<expr> EqualNode <expr>
	\__<expr> PlusNode <expr>
		\__ ConstantNumberNode: 6
		\__ ConstantNumberNode: 7
	\__ ConstantNumberNode: 13


In [29]:
parse("1+2=3")

Los tokens son [int: 1, +: +, int: 2, =: =, int: 3, $: $]
[A -> int, A -> int + A, A -> int, E -> A = A]
-----------------------------
['SHIFT', 'SHIFT', 'SHIFT', 'REDUCE', 'REDUCE', 'SHIFT', 'SHIFT', 'REDUCE', 'REDUCE']
###############################
 
 El ast es: 
 \__<expr> EqualNode <expr>
	\__<expr> PlusNode <expr>
		\__ ConstantNumberNode: 1
		\__ ConstantNumberNode: 2
	\__ ConstantNumberNode: 3
